In [3]:
!pip install lyzr youtube-transcript-api fpdf2 ipywidgets
!apt-get update -qq && apt-get install -y fonts-dejavu-core

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.0/143.0 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 42.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 890.2/890.2 kB 43.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 91.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.3/120.3 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 50.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.7/245.7 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.6/165.6 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  fonts-dejavu-core
0 upgraded, 1 newly installed, 0 to remove and 100 not upgraded.
Need to get 1,041 kB of archives.
After this operation, 3,025 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 fonts-dejavu-core all 2.37-2build1 [1,041 kB]
Fetched 1,041 kB in 1s (1,084 kB/s)
Selecting previously unselected package fonts-dejavu-core.
(Reading database ... 126102 files and directories currently installed.)
Preparing to unpack .../fonts-dejavu-core_2.37-2build1_all.deb ...
Unpacking fonts-dejavu-core (2.37-2build1) ...
Setting up fonts-dejavu-core (2.37-2build1) ...
Processing triggers for fontconfig (2.13.1-4.2u

In [1]:
import os
import openai

openai.api_key = os.getenv("OPENAI_API_KEY")
os.environ['OPENAI_API_KEY'] = "YOUR_OPENAI_API_KEY_HERE"

In [2]:
import json
from lyzr import ChatBot
from youtube_transcript_api import YouTubeTranscriptApi, TranscriptsDisabled, NoTranscriptFound, CouldNotRetrieveTranscript
from fpdf import FPDF
from ipywidgets import Textarea, Button, Output, Layout
from IPython.display import display, Markdown
import re

In [3]:
def transcript_to_pdf(video_id: str, output_pdf_path: str) -> bool:
    """
    Download YouTube transcript (manual or auto) and write it into a PDF
    using the system-installed DejaVuSans.ttf for full Unicode support.
    Fixed to handle long words and text formatting issues.
    """
    try:
        entries = YouTubeTranscriptApi.get_transcript(video_id)
    except (TranscriptsDisabled, NoTranscriptFound, CouldNotRetrieveTranscript):
        try:
            entries = YouTubeTranscriptApi.get_transcript(video_id, languages=['en'])
        except Exception:
            print(f"[!] No transcript for {video_id}")
            return False
    except Exception as e:
        print(f"[!] Error fetching transcript for {video_id}: {e}")
        return False

    text = "\n".join(e['text'] for e in entries).strip()
    if not text:
        print(f"[!] Empty transcript for {video_id}")
        return False

    pdf = FPDF()
    pdf.add_page()

    font_path = "/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf"
    try:
        if os.path.exists(font_path):
            pdf.add_font("DejaVu", "", font_path)
            pdf.set_font("DejaVu", size=10)
        else:
            pdf.set_font("Arial", size=10)
    except Exception:
        pdf.set_font("Arial", size=10)

    pdf.set_margins(20, 20, 20)
    pdf.set_auto_page_break(auto=True, margin=25)

    def process_text_for_pdf(text):
        text = re.sub(r'\s+', ' ', text)
        text = text.replace('\n\n', '\n')

        processed_lines = []
        for paragraph in text.split('\n'):
            if not paragraph.strip():
                continue

            words = paragraph.split()
            processed_words = []
            for word in words:
                if len(word) > 50:
                    chunks = [word[i:i+50] for i in range(0, len(word), 50)]
                    processed_words.extend(chunks)
                else:
                    processed_words.append(word)

            processed_lines.append(' '.join(processed_words))

        return processed_lines

    processed_lines = process_text_for_pdf(text)

    for line in processed_lines:
        if line.strip():
            try:
                pdf.multi_cell(0, 8, line.encode('utf-8', 'replace').decode('utf-8'), align='L')
                pdf.ln(2)
            except Exception as e:
                print(f"[!] Warning: Skipped problematic line: {str(e)[:100]}...")
                continue

    try:
        pdf.output(output_pdf_path)
        print(f"[+] PDF saved: {output_pdf_path}")
        return True
    except Exception as e:
        print(f"[!] Error saving PDF: {e}")
        return False

In [4]:
def create_interactive_chat(agent):
    input_area = Textarea(
        placeholder="Type a question…", layout=Layout(width='80%', height='80px')
    )
    send_button = Button(description="Send", button_style="success")
    output_area = Output(layout=Layout(
        border='1px solid gray', width='80%', height='200px', overflow='auto'
    ))

    def on_send(btn):
        question = input_area.value.strip()
        if not question:
            return
        with output_area:
            print(f">> You: {question}")
            try:
                print("<< Bot:", agent.chat(question), "\n")
            except Exception as e:
                print(f"[!] Error: {e}\n")

    send_button.on_click(on_send)
    display(input_area, send_button, output_area)

In [5]:
def main():
    video_ids = ["dQw4w9WgXcQ", "jNQXAC9IVRw"]
    processed = []

    for vid in video_ids:
        pdf_path = f"{vid}.pdf"
        if transcript_to_pdf(vid, pdf_path):
            processed.append((vid, pdf_path))
        else:
            print(f"[!] Skipping {vid} — no transcript available.")

    if not processed:
        print("[!] No PDFs generated. Please try other video IDs.")
        return

    first_vid, first_pdf = processed[0]
    print(f"[+] Initializing PDF-chat agent for video {first_vid}…")
    bot = ChatBot.pdf_chat(
        input_files=[first_pdf]
    )

    questions = [
        "Summarize the transcript in 2–3 sentences.",
        "What are the top 5 insights and why?",
        "List any recommendations or action items mentioned.",
        "Write 3 quiz questions to test comprehension.",
        "Suggest 5 creative prompts to explore further."
    ]
    responses = {}
    for q in questions:
        print(f"[?] {q}")
        try:
            resp = bot.chat(q)
        except Exception as e:
            resp = f"[!] Agent error: {e}"
        responses[q] = resp
        print(f"[/] {resp}\n" + "-"*60 + "\n")

    with open('responses.json','w',encoding='utf-8') as f:
        json.dump(responses,f,indent=2)
    md = "# Transcript Analysis Report\n\n"
    for q,a in responses.items():
        md += f"## Q: {q}\n{a}\n\n"
    with open('report.md','w',encoding='utf-8') as f:
        f.write(md)

    display(Markdown(md))

    if len(processed) > 1:
        print("[+] Generating comparison…")
        _, pdf1 = processed[0]
        _, pdf2 = processed[1]
        compare_bot = ChatBot.pdf_chat(
            input_files=[pdf1, pdf2]
        )
        comparison = compare_bot.chat(
            "Compare the main themes of these two videos and highlight key differences."
        )
        print("[+] Comparison Result:\n", comparison)

    print("\n=== Interactive Chat (Video 1) ===")
    create_interactive_chat(bot)

In [ ]:
if __name__ == "__main__":
    main()